In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import utils
import plotly.graph_objects as go
from instrument import Instrument

In [2]:
# pair = "EUR_USD"
# pair = "GBP_USD"
# pair = "USD_JPY"
pair = "GBP_JPY"
granularity = "H1"
ma_list = [16,64]
i_pair = Instrument.get_instrument_by_name(pair, True)

In [3]:
i_pair

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'marginRate': '0.0333'}

In [4]:
df = pd.read_pickle(utils.get_data_filename(pair, granularity, True))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [6]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-12-29T19:00:00.000000000Z,179.650,179.776,179.530,179.756,180.109437,180.658203
1,2023-12-29T20:00:00.000000000Z,179.760,179.872,179.695,179.728,180.072313,180.630109
2,2023-12-29T21:00:00.000000000Z,179.728,179.806,179.457,179.550,180.021438,180.599297
3,2024-01-01T22:00:00.000000000Z,179.278,179.490,179.266,179.376,179.958313,180.564203
4,2024-01-01T23:00:00.000000000Z,179.386,179.638,179.279,179.611,179.917812,180.531437


In [7]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [8]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [9]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
43,2024-01-03T14:00:00.000000000Z,180.631,181.055,180.552,180.930,179.904750,179.861938,0.042812,-0.044906,1
121,2024-01-08T20:00:00.000000000Z,183.864,183.924,183.782,183.832,183.550500,183.557750,-0.007250,0.002344,-1
154,2024-01-10T05:00:00.000000000Z,183.944,184.024,183.790,183.902,183.659188,183.623719,0.035469,-0.027187,1
212,2024-01-12T15:00:00.000000000Z,184.562,184.902,184.515,184.676,185.119500,185.164125,-0.044625,0.019063,-1
236,2024-01-15T15:00:00.000000000Z,185.520,185.612,185.370,185.602,185.295875,185.292203,0.003672,-0.044453,1


In [10]:
df_trades['DELTA'] = ( df_trades.mid_c.diff() / i_pair.pipLocation ).shift(-1)
df_trades['GAIN'] = df_trades.DELTA * df_trades.IS_TRADE

In [11]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 43 to 3753
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       65 non-null     object 
 1   mid_o      65 non-null     float64
 2   mid_h      65 non-null     float64
 3   mid_l      65 non-null     float64
 4   mid_c      65 non-null     float64
 5   MA_16      65 non-null     float64
 6   MA_64      65 non-null     float64
 7   DIFF       65 non-null     float64
 8   DIFF_PREV  65 non-null     float64
 9   IS_TRADE   65 non-null     int64  
 10  DELTA      64 non-null     float64
 11  GAIN       64 non-null     float64
dtypes: float64(10), int64(1), object(1)
memory usage: 6.6+ KB


In [12]:
from dateutil.parser import *

df_trades['time'] = [parse(x) for x in df_trades.time]

In [13]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 43 to 3753
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   time       65 non-null     datetime64[ns, tzutc()]
 1   mid_o      65 non-null     float64                
 2   mid_h      65 non-null     float64                
 3   mid_l      65 non-null     float64                
 4   mid_c      65 non-null     float64                
 5   MA_16      65 non-null     float64                
 6   MA_64      65 non-null     float64                
 7   DIFF       65 non-null     float64                
 8   DIFF_PREV  65 non-null     float64                
 9   IS_TRADE   65 non-null     int64                  
 10  DELTA      64 non-null     float64                
 11  GAIN       64 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(10), int64(1)
memory usage: 6.6 KB


In [ ]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)

In [14]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
43,2024-01-03 14:00:00+00:00,180.631,181.055,180.552,180.930,179.904750,179.861938,0.042812,-0.044906,1,290.2,290.2
121,2024-01-08 20:00:00+00:00,183.864,183.924,183.782,183.832,183.550500,183.557750,-0.007250,0.002344,-1,7.0,-7.0
154,2024-01-10 05:00:00+00:00,183.944,184.024,183.790,183.902,183.659188,183.623719,0.035469,-0.027187,1,77.4,77.4
212,2024-01-12 15:00:00+00:00,184.562,184.902,184.515,184.676,185.119500,185.164125,-0.044625,0.019063,-1,92.6,-92.6
236,2024-01-15 15:00:00+00:00,185.520,185.612,185.370,185.602,185.295875,185.292203,0.003672,-0.044453,1,246.2,246.2


In [15]:
df_trades['GAIN'].sum()

np.float64(1223.399999999995)

In [16]:
df_plot = df_ma.iloc[150:300].copy()
df_plot.shape

(150, 10)

In [17]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()